In [1]:
# Imports

import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder


In [2]:
# Upload credit_card_transactions.csv to Colab
from google.colab import files

csv_file = files.upload()

Saving applicants_data.csv to applicants_data (2).csv


In [3]:
# Read the applicants_data.csv file from the Resources folder into a Pandas DataFrame
applicant_data_df = pd.read_csv("applicants_data.csv")

# Review the DataFrame
#applicant_data_df.head()


In [4]:
# Review the data types associated with the columns
# applicant_data_df.dtypes


In [5]:
# Drop the 'EIN' and 'NAME' columns from the DataFrame
applicant_data_df = applicant_data_df.drop(columns=["EIN", "NAME"], axis=1)

# Review the DataFrame
#applicant_data_df.head()


In [6]:
# Create a list of categorical variables 
categorical_variables = list(applicant_data_df.dtypes[applicant_data_df.dtypes =="object"].index)

# Display the categorical variables list
#categorical_variables


In [7]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)


In [8]:
# Encode the categorcal variables using OneHotEncoder
encoded_data = enc.fit_transform(applicant_data_df[categorical_variables])


In [9]:
# Create a DataFrame with the encoded variables
encoded_df = pd.DataFrame(
    encoded_data,
    columns = enc.get_feature_names(categorical_variables)
)

# Review the DataFrame
#encoded_df.head()


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [10]:
#Numerical_Variables

In [11]:
# Add the numerical variables from the original DataFrame to the one-hot encoding DataFrame
encoded_df = pd.concat(
    [
        applicant_data_df[["ASK_AMT", "IS_SUCCESSFUL", "STATUS"]],
        encoded_df
    ],
    axis=1
)

# Review the Dataframe
#encoded_df.head()


In [12]:
# Define the target set y using the IS_SUCCESSFUL column
y = encoded_df["IS_SUCCESSFUL"]

# Display a sample of y
#y[:10]


In [13]:
# Define features set X by selecting all columns but IS_SUCCESSFUL
X = encoded_df.drop(columns=["IS_SUCCESSFUL"], axis=1)

# Review the features DataFrame
#X.head()


In [14]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [15]:
#X_train.dtypes

In [16]:
# Create a StandardScaler instance
scaler = StandardScaler()

In [17]:
# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

In [18]:
# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [19]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])
# Review the number of features
number_input_features


116

In [20]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [21]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 =  (number_input_features + number_output_neurons) // 2

# Review the number hidden nodes in the first layer
hidden_nodes_layer1


58

In [22]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 =  (hidden_nodes_layer1 + number_output_neurons) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2


29

In [23]:
# Create the Sequential model instance
nn = Sequential()


In [24]:
# Add the first hidden layer
nn.add(Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))


In [25]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))


In [26]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons , activation="sigmoid"))


In [27]:
# Display the Sequential model summary
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 58)                6786      
                                                                 
 dense_1 (Dense)             (None, 29)                1711      
                                                                 
 dense_2 (Dense)             (None, 1)                 30        
                                                                 
Total params: 8,527
Trainable params: 8,527
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Fit the model using 50 epochs and the training data
deep_net_model = nn.fit(X_train_scaled, y_train, epochs=25)


Epoch 1/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5742 - accuracy: 0.7199
Epoch 2/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7283
Epoch 3/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5508 - accuracy: 0.7290
Epoch 4/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5490 - accuracy: 0.7316
Epoch 5/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accuracy: 0.7329
Epoch 6/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5454 - accuracy: 0.7348
Epoch 7/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5443 - accuracy: 0.7340
Epoch 8/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5439 - accuracy: 0.7323
Epoch 9/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5429 - accuracy: 0.7324
Epoch 10/25
804/804 [==============================] - 2s 2ms/step - loss: 0.5426 - accuracy: 0.7352

In [30]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5518 - accuracy: 0.7324 - 456ms/epoch - 2ms/step
Loss: 0.5518364310264587, Accuracy: 0.7323614954948425


In [31]:
# Set the model's file path
file_path = "Alt_AlphabetSoup.h5"

# Export your model to a HDF5 file
nn.save(file_path)


### Alternative Model 1

In [32]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [33]:
# Define the number of neurons in the output layer
number_output_neurons_A1 = 3

In [34]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A1 = (number_input_features + number_output_neurons_A1) // 2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A1

59

In [35]:
# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2_A1 =  (hidden_nodes_layer1_A1 + number_output_neurons) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer2_A1

30

In [36]:
# Define the number of hidden nodes for the third hidden layer
hidden_nodes_layer3_A1 =  (hidden_nodes_layer2_A1 + number_output_neurons) // 2

# Review the number hidden nodes in the second layer
hidden_nodes_layer3_A1

15

In [37]:
# Create the Sequential model instance
nn_A1 = Sequential()

In [38]:
# First hidden layer
nn_A1.add(Dense(units = hidden_nodes_layer1_A1, input_dim = number_input_features, activation = "relu"))

# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2_A1, activation="relu"))

# Add the third hidden layer
nn.add(Dense(units=hidden_nodes_layer3_A1, activation="relu"))

# Output layer
nn_A1.add(Dense(units = number_output_neurons_A1, activation = "linear"))

# Check the structure of the model
nn_A1.summary

<bound method Model.summary of <keras.engine.sequential.Sequential object at 0x7fbf131a9110>>

In [39]:
# Compile the Sequential model
nn_A1.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])


In [40]:
# Fit the model using 50 epochs and the training data
fit_model_A1 = nn_A1.fit(X_train_scaled,y_train,epochs=50)


Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 1.9838 - accuracy: 0.3869
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 1.1910 - accuracy: 0.3027
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 1.2104 - accuracy: 0.3117
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 1.1939 - accuracy: 0.3316
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 1.0746 - accuracy: 0.3470
Epoch 6/50
804/804 [==============================] - 2s 2ms/step - loss: 1.1146 - accuracy: 0.3616
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 1.1524 - accuracy: 0.3449
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 1.1894 - accuracy: 0.3378
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 1.0723 - accuracy: 0.3185
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 1.0530 - accuracy: 0.3277

#### Alternative Model 2

In [55]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
number_input_features

116

In [56]:
# Define the number of neurons in the output layer
number_output_neurons_A2 = 2

In [54]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1_A2 = (number_input_features + number_output_neurons_A1) // 2

# Review the number of hidden nodes in the first layer
hidden_nodes_layer1_A2

# Define the number of hidden nodes for the 2nd hidden layer
hidden_nodes_layer1_A2 = (hidden_nodes_layer1_A2 + number_output_neurons_A1) // 2

# Review the number of hidden nodes in the first layer
#hidden_nodes_layer1_A2

31

In [57]:
# Create the Sequential model instance
nn_A2 =  Sequential()

In [61]:
# First hidden layer
nn_A2.add(Dense(units = hidden_nodes_layer1_A2, input_dim = number_input_features, activation = "relu"))


# Output layer
nn_A2.add(Dense(number_output_neurons_A2, activation = "linear")
)


# Check the structure of the model
nn_A2.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_13 (Dense)            (None, 31)                3627      
                                                                 
 dense_14 (Dense)            (None, 2)                 64        
                                                                 
 dense_15 (Dense)            (None, 31)                93        
                                                                 
 dense_16 (Dense)            (None, 2)                 64        
                                                                 
Total params: 3,848
Trainable params: 3,848
Non-trainable params: 0
_________________________________________________________________


In [62]:
# Compile the model
nn_A2.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])


In [63]:
# Fit the model
fit_model_A2 = nn_A2.fit(X_train_scaled,y_train,epochs=100)


Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 1.2040 - accuracy: 0.6370
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5690 - accuracy: 0.6372
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5589 - accuracy: 0.5786
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5549 - accuracy: 0.5536
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5523 - accuracy: 0.5341
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5501 - accuracy: 0.5329
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.4899
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5483 - accuracy: 0.4691
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5474 - accuracy: 0.5234
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5463 - accura

### After finishing your models, display the accuracy scores achieved by each model, and compare the results.

In [64]:
print("Original Model Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose = 2)


# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Original Model Results
268/268 - 0s - loss: 0.5518 - accuracy: 0.7324 - 328ms/epoch - 1ms/step
Loss: 0.5518364310264587, Accuracy: 0.7323614954948425


In [65]:
print("Alternative Model 3 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A1.evaluate(X_test_scaled, y_test, verbose = 2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 3 Results
268/268 - 0s - loss: 0.7387 - accuracy: 0.4412 - 436ms/epoch - 2ms/step
Loss: 0.7386655807495117, Accuracy: 0.4411661922931671


In [66]:
print("Alternative Model 4 Results")

# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_A2.evaluate(X_test_scaled, y_test, verbose = 2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Alternative Model 4 Results
268/268 - 0s - loss: 0.5506 - accuracy: 0.5569 - 455ms/epoch - 2ms/step
Loss: 0.5506153702735901, Accuracy: 0.556851327419281


### Step 3: Save each of your alternative models as an HDF5 file.


In [67]:
# Set the file path for the first alternative model
file_path = "Alt_Model1.h5"

# Export your model to a HDF5 file
nn.save(file_path)


In [68]:
# Set the file path for the second alternative model
file_path = "Alt_Model2.h5"

# Export your model to a HDF5 file
nn.save(file_path)
